In [1]:
from typing import List, Tuple, Set
from collections import deque
import heapq
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

class SearchAlgorithm:
    """A class implementing various search algorithms for grid-based pathfinding."""

    @staticmethod
    def get_neighbors(x: int, y: int, grid: List[List[str]]) -> List[Tuple[int, int]]:
        """Find valid adjacent cells (up, down, left, right) that aren't obstacles."""
        rows, cols = len(grid), len(grid[0])
        neighbors = []
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < rows and 0 <= new_y < cols and grid[new_x][new_y] != '-1':
                neighbors.append((new_x, new_y))

        return neighbors

    @staticmethod
    def get_heuristic(x: int, y: int, target_x: int, target_y: int) -> int:
        """Calculate Manhattan distance heuristic from position to target."""
        return abs(target_x - x) + abs(target_y - y)

    @staticmethod
    def get_start_target(grid: List[List[str]]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        """Find the start ('s') and target ('t') positions in the grid."""
        start, target = None, None
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if grid[row][col] == "s":
                    start = (row, col)
                elif grid[row][col] == "t":
                    target = (row, col)
                
                # Break early if both found
                if start is not None and target is not None:
                    return start, target
                    
        # If either start or target is missing
        if start is None or target is None:
            return None, None
            
        return start, target

    @staticmethod
    def reconstruct_path(parents: dict, start: Tuple[int, int], target: Tuple[int, int]) -> List[Tuple[int, int]]:
        """Reconstruct the path from start to target using the parents dictionary."""
        if target not in parents and target != start:
            return []  # No path exists
            
        path = []
        current = target
        
        # Build path from target back to start
        while current in parents:
            path.append(current)
            current = parents[current]
            
        path.append(start)  # Add the start node
        path.reverse()      # Reverse to get start-to-target order
        
        return path

    @staticmethod
    def bfs(grid: List[List[str]]) -> Tuple[int, List[Tuple[int, int]], List[Tuple[int, int]]]:
        """Breadth-First Search algorithm to find shortest path."""
        start, target = SearchAlgorithm.get_start_target(grid)
        if start is None or target is None:
            return -1, [], []

        queue = deque([(start, 0)])  # (node, distance)
        visited = {start}
        parents = {}
        traversal = []

        while queue:
            current, distance = queue.popleft()
            traversal.append(current)
            
            if current == target:
                path = SearchAlgorithm.reconstruct_path(parents, start, target)
                return distance, path, traversal

            for neighbor in SearchAlgorithm.get_neighbors(*current, grid):
                if neighbor not in visited:
                    queue.append((neighbor, distance + 1))
                    visited.add(neighbor)
                    parents[neighbor] = current

        return -1, [], traversal  # No path found

    @staticmethod
    def dfs(grid: List[List[str]]) -> Tuple[int, List[Tuple[int, int]], Set[Tuple[int, int]]]:
        """Depth-First Search algorithm to find a path (not necessarily shortest)."""
        start, target = SearchAlgorithm.get_start_target(grid)
        if start is None or target is None:
            return -1, [], set()

        stack = deque([(start, 0)])  # (node, distance)
        visited = {start}
        parents = {}
        traversal = []

        while stack:
            current, distance = stack.pop()  # Pop from the end (right side) for DFS
            traversal.append(current)
            
            if current == target:
                path = SearchAlgorithm.reconstruct_path(parents, start, target)
                return distance, path, traversal

            for neighbor in SearchAlgorithm.get_neighbors(*current, grid):
                if neighbor not in visited:
                    stack.append((neighbor, distance + 1))
                    visited.add(neighbor)
                    parents[neighbor] = current

        return -1, [], traversal  # No path found

    @staticmethod
    def ucs(grid: List[List[str]]) -> Tuple[int, List[Tuple[int, int]], List[Tuple[int, int]]]:
        """Uniform Cost Search algorithm to find shortest path."""
        start, target = SearchAlgorithm.get_start_target(grid)
        if start is None or target is None:
            return -1, [], []

        priority_queue = [(0, start)]  # (cost, node)
        visited = set()
        parents = {}
        distances = {start: 0}
        traversal = []

        while priority_queue:
            cost, current = heapq.heappop(priority_queue)
            
            if current in visited:
                continue
                
            visited.add(current)
            traversal.append(current)
            
            if current == target:
                path = SearchAlgorithm.reconstruct_path(parents, start, target)
                return cost, path, traversal

            for neighbor in SearchAlgorithm.get_neighbors(*current, grid):
                new_cost = cost + 1  # Assume uniform edge costs of 1
                
                if neighbor not in distances or new_cost < distances[neighbor]:
                    distances[neighbor] = new_cost
                    parents[neighbor] = current
                    heapq.heappush(priority_queue, (new_cost, neighbor))

        return -1, [], traversal  # No path found

    @staticmethod
    def best_first_search(grid: List[List[str]]) -> Tuple[int, List[Tuple[int, int]], List[Tuple[int, int]]]:
        """Best-First Search (Greedy) algorithm using heuristic only."""
        start, target = SearchAlgorithm.get_start_target(grid)
        if start is None or target is None:
            return -1, [], []

        # Initialize with heuristic value
        priority_queue = [(SearchAlgorithm.get_heuristic(*start, *target), start)]
        visited = set()
        parents = {}
        distances = {start: 0}
        traversal = []

        while priority_queue:
            _, current = heapq.heappop(priority_queue)
            
            if current in visited:
                continue
                
            visited.add(current)
            traversal.append(current)
            
            if current == target:
                path = SearchAlgorithm.reconstruct_path(parents, start, target)
                return distances[current], path, traversal

            for neighbor in SearchAlgorithm.get_neighbors(*current, grid):
                if neighbor not in visited:
                    new_distance = distances[current] + 1
                    distances[neighbor] = new_distance
                    parents[neighbor] = current
                    # Only use heuristic for priority
                    heapq.heappush(priority_queue, (SearchAlgorithm.get_heuristic(*neighbor, *target), neighbor))

        return -1, [], traversal  # No path found

    @staticmethod
    def a_star_search(grid: List[List[str]]) -> Tuple[int, List[Tuple[int, int]], List[Tuple[int, int]]]:
        """A* Search algorithm combining path cost and heuristic."""
        start, target = SearchAlgorithm.get_start_target(grid)
        if start is None or target is None:
            return -1, [], []
            
        priority_queue = [(SearchAlgorithm.get_heuristic(*start, *target), 0, start)]  # (f_score, g_score, node)
        visited = set()
        parents = {}
        g_scores = {start: 0}  # Cost from start to node
        traversal = []

        while priority_queue:
            _, g_score, current = heapq.heappop(priority_queue)
            
            if current in visited:
                continue
                
            visited.add(current)
            traversal.append(current)
            
            if current == target:
                path = SearchAlgorithm.reconstruct_path(parents, start, target)
                return g_score, path, traversal

            for neighbor in SearchAlgorithm.get_neighbors(*current, grid):
                new_g_score = g_score + 1
                
                if neighbor not in g_scores or new_g_score < g_scores[neighbor]:
                    g_scores[neighbor] = new_g_score
                    parents[neighbor] = current
                    f_score = new_g_score + SearchAlgorithm.get_heuristic(*neighbor, *target)
                    heapq.heappush(priority_queue, (f_score, new_g_score, neighbor))

        return -1, [], traversal  # No path found

    @staticmethod
    def mark_path(grid: List[List[str]], path: List[Tuple[int, int]]) -> List[List[str]]:
        """Mark the path on a copy of the grid with step numbers."""
        grid_copy = [row.copy() for row in grid]
        
        # Skip start and target nodes for step numbering
        step = 1
        for x, y in path:
            if grid_copy[x][y] not in ['s', 't']:
                grid_copy[x][y] = str(step)
                step += 1
                
        return grid_copy

    @staticmethod
    def visualize_grid(grid: List[List[str]], path: List[Tuple[int, int]], visited: List[Tuple[int, int]], output_file: str):
        """Visualize the grid, path, and visited nodes."""
        if not path:
            print(f"No valid path found for {output_file}, skipping visualization.")
            return

        # Create a copy with path markers
        grid_copy = SearchAlgorithm.mark_path(grid, path)
        
        rows = len(grid_copy)
        cols = len(grid_copy[0])
        fig, ax = plt.subplots(figsize=(cols * 1.2, rows * 1.2))
        ax.set_xlim(0, cols)
        ax.set_ylim(0, rows)
        ax.invert_yaxis()
        ax.axis('off')

        # Draw the grid
        for i in range(rows):
            for j in range(cols):
                cell = grid_copy[i][j]
                
                # Set the cell color
                if cell == 's':
                    facecolor = "green"
                elif cell == 't':
                    facecolor = "red"
                elif cell == '-1':
                    facecolor = "lightgray"
                else:
                    facecolor = "white"
                
                # Draw cell rectangle
                rect = Rectangle((j, i), 1, 1, facecolor=facecolor, edgecolor="black")
                ax.add_patch(rect)
                
                # Add cell text
                ax.text(j + 0.5, i + 0.5, cell, ha="center", va="center", fontsize=14)

        # Draw the path
        if path:
            line_x = [col + 0.5 for (row, col) in path]
            line_y = [row + 0.5 for (row, col) in path]
            ax.plot(line_x, line_y, color="red", linewidth=3, marker="o", markersize=5)

        plt.savefig(output_file, bbox_inches='tight')
        plt.close(fig)
        print(f"Grid visualization saved to {output_file}")


def test_algorithms(grid: List[List[str]], name: str):
    """Test all search algorithms on a grid and visualize the results."""
    algorithms = {
        "BFS": SearchAlgorithm.bfs,
        "DFS": SearchAlgorithm.dfs,
        "UCS": SearchAlgorithm.ucs,
        "Best-First": SearchAlgorithm.best_first_search,
        "A*": SearchAlgorithm.a_star_search
    }
    
    for algo_name, algo_func in algorithms.items():
        # Create a fresh copy of the grid for each algorithm
        grid_copy = [row.copy() for row in grid]
        print(f"\n{algo_name} on {name}")
        
        # Run the algorithm
        distance, path, traversal = algo_func(grid_copy)
        
        if distance == -1:
            print(f"No path found")
        else:
            print(f"Shortest Distance: {distance}")
            print(f"Path Length: {len(path)}")
            
            # Visualize the results
            output_file = f"{name}_{algo_name.replace('*', 'star')}.png"
            SearchAlgorithm.visualize_grid(grid_copy, path, traversal, output_file)
            
            # Print traversal details
            print(f"Nodes explored: {len(traversal)}")


if __name__ == "__main__":
    # Example grids
    examples = {
        "example_1": [
            ['s', '1', '2', '11', '14'],
            ['-1', '-1', '3', '8', '12'],
            ['t', '-1', '4', '5', '9'],
            ['-1', '7', '6', '-1', '13'],
            ['0', '-1', '10', '-1', '15']
        ],
        
        "complex_grid": [
            ['s', '1', '3', '5', '-1', '-1', '-1', '-1'],
            ['2', '4', '6', '7', '9', '-1', '-1', '-1'],
            ['-1', '-1', '8', '-1', '-1', '16', '0', '0'],
            ['-1', '12', '10', '11', '13', '14', '15', 't']
        ],
        
        "maze_grid": [
             ['s', '1', '3', '-1', '0'],
             ['2', '-1', '5', '-1', 't'],
             ['4', '-1', '7', '10', '12'],
             ['6', '8', '11', '-1', '13'],
             ['9', '-1', '-1', '-1', '0']
        ]
    }
    
    # Test each grid with all algorithms
    for name, grid in examples.items():
        test_algorithms(grid, name)


BFS on example_1
No path found

DFS on example_1
No path found

UCS on example_1
No path found

Best-First on example_1
No path found

A* on example_1
No path found

BFS on complex_grid
Shortest Distance: 10
Path Length: 11
Grid visualization saved to complex_grid_BFS.png
Nodes explored: 18

DFS on complex_grid
Shortest Distance: 12
Path Length: 13
Grid visualization saved to complex_grid_DFS.png
Nodes explored: 16

UCS on complex_grid
Shortest Distance: 10
Path Length: 11
Grid visualization saved to complex_grid_UCS.png
Nodes explored: 19

Best-First on complex_grid
Shortest Distance: 12
Path Length: 13
Grid visualization saved to complex_grid_Best-First.png
Nodes explored: 14

A* on complex_grid
Shortest Distance: 10
Path Length: 11
Grid visualization saved to complex_grid_Astar.png
Nodes explored: 16

BFS on maze_grid
Shortest Distance: 7
Path Length: 8
Grid visualization saved to maze_grid_BFS.png
Nodes explored: 15

DFS on maze_grid
Shortest Distance: 9
Path Length: 10
Grid visua